In [10]:
import numpy as np
import fasttext.util

#ft = fasttext.load_model('cc.en.300.bin')
#fasttext.util.reduce_model(ft, 50)
#ft.get_dimension()
#
#print(ft.get_word_vector('hello'))

In [14]:
data = ["the cat likes oranges"]
target = ["le chat aime les oranges"]

class TranslationDataset():
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return (len(self.images))

    def __getitem__(self, idx):
        return({
            "image":self.images[idx],
            "label":int(self.labels[idx]),
        })

    def rand_sample(self, n):
        r = random.randint(0, len(self.images))

        output = []
        for i in range(r, r + n):
            idx = (i % len(self.images))
            output.append({
                "image":self.images[idx],
                "label":int(self.labels[idx]),
            })
        return output
    

dataset = TranslationDataset(data, target) 

print(dataset[0])

ValueError: invalid literal for int() with base 10: 'le chat aime les oranges'